### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk

In [1]:
import json
import nltk
import numpy as np

In [2]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [3]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [4]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    #print(summary_str)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
    ne_index_list = []
    for ne in main_list:
        ne_idx = case_content.find(ne)
        if ne_idx != -1 :
            ne_index_list.append((ne_idx,len(ne)))
    '''
    if len(main_list) != len(ne_index_list):
        c += 1
        print(case_content)
        print(main_list)
        for i,l in ne_index_list:
            print(case_content[i])
        print("------------")
    '''
    
    if len(main_list) == len(ne_index_list):
        texts = []
        for i,w in enumerate(case_content):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))
        
        docs.append(texts)

In [5]:
print(len(docs))

1436


In [ ]:
### 計算平均句長
total_wc = 0
for doc in docs:
    total_wc += len(doc)
    
ave = total_wc / 2701
print(ave)

In [6]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    tagged = nltk.pos_tag(tokens)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

print(data[0])

[('年', 'JJ', 'O'), ('前', 'NNP', 'O'), ('治', 'NNP', 'O'), ('一', 'NNP', 'O'), ('张', 'NNP', 'O'), ('姓', 'NNP', 'O'), ('中', 'NNP', 'O'), ('年', 'NNP', 'O'), ('男', 'NNP', 'O'), ('子', 'NNP', 'O'), ('，', 'NNP', 'O'), ('咳', 'NNP', 'O'), ('嗽', 'NNP', 'O'), ('寒', 'NNP', 'O'), ('冷', 'NNP', 'O'), ('发', 'NNP', 'O'), ('烧', 'NNP', 'O'), ('，', 'NNP', 'O'), ('胸', 'NNP', 'O'), ('部', 'NNP', 'O'), ('摄', 'NNP', 'O'), ('片', 'NNP', 'O'), ('为', 'NNP', 'O'), ('“', 'NNP', 'O'), ('肺', 'NNP', 'O'), ('炎', 'NNP', 'O'), ('”', 'NNP', 'O'), ('，', 'NNP', 'O'), ('经', 'NNP', 'O'), ('输', 'NNP', 'O'), ('液', 'NNP', 'O'), ('烧', 'NNP', 'O'), ('退', 'NNP', 'O'), ('咳', 'NNP', 'O'), ('减', 'NNP', 'O'), ('。', 'NNP', 'O'), ('但', 'NNP', 'O'), ('仍', 'NNP', 'O'), ('咳', 'NNP', 'B-S'), ('吐', 'NNP', 'I-S'), ('稀', 'NNP', 'I-S'), ('痰', 'NNP', 'E-S'), ('、', 'NNP', 'O'), ('恶', 'NNP', 'B-S'), ('风', 'NNP', 'E-S'), ('，', 'NNP', 'O'), ('自', 'NNP', 'O'), ('觉', 'NNP', 'O'), ('阴', 'NNP', 'B-S'), ('囊', 'NNP', 'I-S'), ('湿', 'NNP', 'I-S'), ('冷', 'NNP', 

In [9]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]
    
    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [10]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [11]:
print(len(X_train[0][4]))
print(X_train[0][5])

18
['bias', 'word.lower=，', 'word[-3:]=，', 'word[-2:]=，', 'word.isupper=False', 'word.istitle=False', 'word.isdigit=False', 'postag=NNP', '-1:word.lower=某', '-1:word.istitle=False', '-1:word.isupper=False', '-1:word.isdigit=False', '-1:postag=NNP', '+1:word.lower=女', '+1:word.istitle=False', '+1:word.isupper=False', '+1:word.isdigit=False', '+1:postag=NNP']


In [12]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 27263
Seconds required: 1.969

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 148956.127459
Feature norm: 1.000000
Error norm: 67573.185052
Active features: 27156
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 0.721

***** Iteration #2 *****
Loss: 147343.476740
Feature norm: 1.017906
Error norm: 57047.216762
Active features: 25979
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.374

***** Iteration #3 *****
Loss: 141208.993677
Feature norm: 1.111373
Error norm: 38791.121317
Active features: 20062
Line search trials: 1
Line search step: 1.000000
Seconds required 

***** Iteration #39 *****
Loss: 28149.070325
Feature norm: 22.661603
Error norm: 24883.416112
Active features: 23072
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.372

***** Iteration #40 *****
Loss: 27699.015798
Feature norm: 22.967846
Error norm: 29178.668469
Active features: 23036
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.372

***** Iteration #41 *****
Loss: 27187.892416
Feature norm: 23.380242
Error norm: 26006.269519
Active features: 22984
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.372

***** Iteration #42 *****
Loss: 26780.835175
Feature norm: 23.610535
Error norm: 16026.302768
Active features: 22969
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #43 *****
Loss: 26367.396697
Feature norm: 23.869237
Error norm: 6563.878499
Active features: 22956
Line search trials: 1
Line search step: 1.00

***** Iteration #86 *****
Loss: 17336.954463
Feature norm: 69.192360
Error norm: 1876.512323
Active features: 19710
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.372

***** Iteration #87 *****
Loss: 17287.068243
Feature norm: 70.623327
Error norm: 4466.869657
Active features: 19543
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #88 *****
Loss: 17238.999071
Feature norm: 71.352658
Error norm: 2483.244355
Active features: 19578
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.385

***** Iteration #89 *****
Loss: 17196.204424
Feature norm: 72.570616
Error norm: 5520.869825
Active features: 19532
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.370

***** Iteration #90 *****
Loss: 17154.974926
Feature norm: 73.305707
Error norm: 2959.335556
Active features: 19417
Line search trials: 1
Line search step: 1.000000

***** Iteration #129 *****
Loss: 16600.636116
Feature norm: 88.911887
Error norm: 4316.049148
Active features: 18519
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.375

***** Iteration #130 *****
Loss: 16591.955354
Feature norm: 89.062801
Error norm: 2451.115507
Active features: 18527
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #131 *****
Loss: 16589.248732
Feature norm: 89.383578
Error norm: 4518.731983
Active features: 18509
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.375

***** Iteration #132 *****
Loss: 16579.532064
Feature norm: 89.519464
Error norm: 1944.914231
Active features: 18510
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.369

***** Iteration #133 *****
Loss: 16577.188442
Feature norm: 89.774055
Error norm: 4263.284104
Active features: 18504
Line search trials: 1
Line search step: 1.0

***** Iteration #175 *****
Loss: 16438.418091
Feature norm: 95.842245
Error norm: 1694.128999
Active features: 18142
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.739

***** Iteration #176 *****
Loss: 16435.056948
Feature norm: 95.893946
Error norm: 1499.664920
Active features: 18153
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.745

***** Iteration #177 *****
Loss: 16432.399380
Feature norm: 95.993673
Error norm: 1553.662959
Active features: 18137
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.746

***** Iteration #178 *****
Loss: 16428.994265
Feature norm: 96.040846
Error norm: 1348.743877
Active features: 18140
Line search trials: 2
Line search step: 0.500000
Seconds required for this iteration: 0.738

***** Iteration #179 *****
Loss: 16428.607554
Feature norm: 96.229571
Error norm: 3874.065029
Active features: 18096
Line search trials: 1
Line search step: 1.0

In [13]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

赵 (O)
某 (O)
， (O)
男 (O)
， (O)
5 (O)
6 (O)
岁 (O)
。 (O)
有 (O)
冠 (O)
心 (O)
病 (O)
史 (O)
已 (O)
3 (O)
年 (O)
， (O)
但 (O)
症 (O)
状 (O)
轻 (O)
， (O)
偶 (O)
有 (O)
心 (O)
悸 (O)
， (O)
胸 (B-S)
闷 (I-S)
痞 (I-S)
塞 (E-S)
， (O)
仍 (O)
坚 (O)
持 (O)
办 (O)
公 (O)
室 (O)
工 (O)
作 (O)
。 (O)
2 (O)
个 (O)
月 (O)
来 (O)
又 (O)
患 (O)
急 (O)
性 (O)
支 (O)
气 (O)
管 (O)
炎 (O)
， (O)
咳 (O)
嗽 (O)
时 (O)
作 (O)
， (O)
咯 (B-S)
吐 (I-S)
白 (I-S)
沫 (I-S)
痰 (E-S)
， (O)
胸 (O)
中 (O)
痞 (O)
塞 (O)
较 (O)
前 (O)
加 (O)
重 (O)
， (O)
纳 (O)
食 (O)
略 (O)
减 (O)
， (O)
大 (O)
便 (O)
尚 (O)
调 (O)
， (O)
下 (O)
肢 (O)
微 (O)
肿 (O)
， (O)
小 (O)
便 (O)
量 (O)
减 (O)
， (O)
舌 (O)
淡 (O)
苔 (O)
薄 (O)
白 (O)
， (O)
脉 (O)
滑 (O)
小 (O)
数 (O)
。 (O)
证 (O)
属 (O)
心 (O)
阳 (O)
不 (O)
振 (O)
， (O)
痰 (O)
饮 (O)
内 (O)
结 (O)
之 (O)
胸 (O)
痹 (O)
。 (O)
治 (O)
用 (O)
茯 (O)
苓 (O)
杏 (O)
仁 (O)
甘 (O)
草 (O)
汤 (O)
合 (O)
二 (O)
陈 (O)
汤 (O)
以 (O)
宣 (O)
肺 (O)
化 (O)
饮 (O)
， (O)
处 (O)
方 (O)
： (O)
茯 (O)
苓 (O)
3 (O)
0 (O)
g (O)
， (O)
陈 (O)
皮 (O)
1 (O)
0 (O)
g (O)
， (O)
制 (O)
半 (O)
夏 (O)
1 (O)
0 (O)
g (O)
， (O)
杏 (O)
仁 (O

In [14]:
from sklearn.metrics import accuracy_score

# Convert the sequences of tags into a 1-dimensional array
#predictions = np.array([labels[tag] for row in y_pred for tag in row])
#truths = np.array([labels[tag] for row in y_test for tag in row])

print(y_pred)

#print(accuracy_score(y_test, y_pred))

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'], ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-S', 'I-S', 'I-S', 'I-S', 'E-S', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-S', 'I-S', 'I

In [15]:
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-SYM","I-SYM","E-SYM"]))

             precision    recall  f1-score   support

          O       0.98      0.99      0.98    209559
      B-SYM       0.48      0.20      0.28      1775
      I-SYM       0.40      0.22      0.28      2815
      E-SYM       0.49      0.21      0.29      1765

avg / total       0.96      0.97      0.96    215914

